# Demos for analyzing World Color Survey (WCS)

COG 260: Data, Computation, and The Mind (Yang Xu)

Data source: http://www1.icsi.berkeley.edu/wcs/data.html

______________________________________________

Import helper function file for WCS data analysis.

In [ ]:
from wcs_helper_functions import *

Import relevant Python libraries.

In [ ]:
import numpy as np
from scipy import stats
from scipy import spatial  
from random import random
from string import ascii_lowercase
from sklearn.model_selection import LeaveOneOut
%matplotlib inline

## Demo 1: Import stimulus (color chip) information in [Munsell space](https://en.wikipedia.org/wiki/Munsell_color_system)

> Stimuli were 330 color chips in Munsell space, each defined along lightness and hue dimensions.

> Each color chip has an index _(from 1 to 330)_ and a coordinate (lightness *(alphabet)*, hue *(integer)*).

In the following section, you will learn how to convert from **(a) index to coordinate** and **(b) coordinate to index**.

______________________________________________

Load chip information in Munsell space. 

`munsellInfo` is a 2-element tuple with dictionary elements.

In [ ]:
munsellInfo = readChipData('./WCS_data_core/chip.txt');

### (a) Index &rarr; Coordinate

Access the second dictionary in `munsellInfo`.

In [ ]:
indexCoord = munsellInfo[1]

`indexCoord` is a dictionary with **index _(key)_ &rarr; coordinate _(value)_** pairs. For example, to retrieve the Munsell coordinate _(lightness, hue)_ for chip with numerical index 1:

In [ ]:
print(indexCoord[1])

You can also uncomment the following to display full stimulus information (long).

In [ ]:
# print(indexCoord)

### (b) Coordinate &rarr; Index

Access the first dictionary in `munsellInfo`.

In [ ]:
coordIndex = munsellInfo[0]

`coordIndex` is a dictionary with **coordinate _(key)_ &rarr; index _(value)_** pairs. For example, to access numerical index for the color chip at Munsell coordinate _(D, 11)_:

In [ ]:
print(coordIndex['D11'])

You can also uncomment the following to display full stimulus information (long).

In [ ]:
# print(coordIndex)

## Demo 2: Import stimulus information in [CIELAB space](https://en.wikipedia.org/wiki/Lab_color_space)

> Each of the 330 stimuli can also be mapped to the 3D CIELAB space, where the dimensions are `l` _(lightness)_, `a`, & `b` _(color opponency)_.

> CIELAB Coordinates have a **one-to-one** correspondence with Munsell index, which ranges from 1 to 330.

______________________________________________

Load chip coordinates in CIELAB. 

`cielabCoord` is a dictionary with **index _(key)_ &rarr; CIELAB Coordinate _(value)_** pairs.

In [ ]:
cielabCoord = readClabData('./WCS_data_core/cnum-vhcm-lab-new.txt')

For example, to obtain the CIELAB coordinates for chip with numerical index 1:

In [ ]:
print(cielabCoord[1])

## Demo 3: Import color naming data
    
> Each of the 330 color chips was named by speakers of 110 different languages.

______________________________________________

Load naming data. 

`namingData` is a hierarchical dictionary organized as follows:

**language _(1 - 110)_ &rarr; speaker _(1 - *range varies per language*)_ &rarr; chip index _(1 - 330)_ &rarr; color term**

In [ ]:
namingData = readNamingData('./WCS_data_core/term.txt')

For example, to obtain naming data from language 1 and speaker 1 for all 330 color chips:

In [ ]:
namingData[1][1]; # remove semicolon to see data in full

For example, to see how many speakers language 1 has:

In [ ]:
len(namingData[1])

## Demo 4: Import color foci data
    
> Apart from naming the color chips, each speaker also pointed to foci color chips for each color term they had used.

> **Note**: A single color term may have multiple foci locations.

______________________________________________

Load foci data. 

`fociData` is a hierarchical dictionary organized as follows: 

**language _(1 - 110)_ &rarr; speaker _(1 - *range varies per language*)_ &rarr; color term &rarr; foci coordinates**

In [ ]:
fociData = readFociData('./WCS_data_core/foci-exp.txt');

For example, to obtain foci data for language 1 and speaker 1, where each entry shows foci locations for given term: 

In [ ]:
fociData[1][1]

In the above example, foci for term 'LF' is located at coordinate _(A, 0)_ in the Munsell chart.

## Demo 5: Import speaker demographic information

> Most speakers' age _(integer)_ and gender _(M/F)_ information was recorded.

______________________________________________

Load speaker information.

`speakerInfo` is a hierarchical dictionary organized as follows:

**language &rarr; speaker &rarr; (age, gender)**

In [ ]:
speakerInfo = readSpeakerData('./WCS_data_core/spkr-lsas.txt')

For example, uncomment the following line to access _(age, gender)_ information for all speakers from language 1:

In [ ]:
# speakerInfo[1]

For example, uncomment the following line to access _(age, gender)_ information for speaker 1 from language 1:

In [ ]:
# speakerInfo[1][1]

## Demo 6: Visualize color naming from an individual speaker

> Naming patterns from a speaker can be visualized in the stimulus palette _(Munsell space)_.

______________________________________________

Extract an example speaker datum from an example language.

In [ ]:
lg61_spk5 = namingData[61][5]

Extract color terms used by that speaker.

In [ ]:
terms = lg61_spk5.values()

Encode the color terms into random numbers (for plotting purposes).

In [ ]:
encoded_terms = map_array_to(terms, generate_random_values(terms))

Visualize the color naming pattern for that speaker&mdash;each color patch corresponds to extension of a color term. Color scheme is randomized, but the partition of the color space is invariant.

In [ ]:
plotValues(encoded_terms)

**Note**: `plotValues()` is a generic function for visualizing various kinds of information on the chart, suited to needs.

Now you are in a position to start exploring this data set - enjoy!

Here we aggregate the data to develop the labels and set up our feature matrix.

In [ ]:

def is_empty(any_structure):
    if any_structure:
        return False
    else:
        return True

### Exemplar Model

def LOOCV_Exemplar(F, terms_inds):
    loo = LeaveOneOut()
    loo.get_n_splits(F)
    accuracy = 0
    prediction_values = []

    for train_index, test_index in loo.split(F):

        # Get training and testing data points

        F_train, F_test = np.asarray(F)[train_index], np.asarray(F)[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # This is the test coordinate (F_test[0]) and the target color term (y_test)
        # print (F_test[0], y_test)

        prediction = ''

        dist_dict = {}
        sim_dict = {}

        for term in color_terms:
            # Calculate the distance from the test coordinate to every single exemplar of each color category
            dist_dict[term] = np.array([spatial.distance.euclidean(coord, F_test[0]) for coord in terms_inds[term] if np.all(coord != F_test[0])])

            # Calculate the average simlarity between the test coordinate and each color category  
            sim_dict[term] = np.sum(np.exp(-(dist_dict[term])**2)) / len(dist_dict[term])

        # print(sim_dict)

        # Choose the color category with the highest similarity 
        prediction = max(sim_dict, key=sim_dict.get)

        # print (prediction)

        if prediction == y_test:
            accuracy += 1 
            
        prediction_values.append(prediction)
    
    accuracy = accuracy / 330
    encoded_terms = map_array_to(prediction_values, generate_random_values(prediction_values))
    plotValues(encoded_terms)
    
    return accuracy
    
# REMEMBER TO DO VISUALS

In [ ]:
acc_list = []
namingData = readNamingData('./WCS_data_core/term.txt')

for i in range (1, 111):    
    ground_truth = {}
    ground_truth_data = {}
    

    #doing lanugage 1. This would eventually need to be changed into a for-loop to cycle through every language
    language = namingData[i]
    print ("=====================STARTING LANGUAGE: ", i, "================================")
    for k in language[1].keys():
      # Aggregate all given color terms for each chip   
      ground_truth_data[k] = tuple(d[1][k] for d in language.items() if d[1][k] != "*")
      if is_empty(ground_truth_data[k]):
        ground_truth_data[k] = '*'
    
    for k in ground_truth_data:
        # choose the color term that appears most frequently as our ground truth value for each chip.
        ground_truth[k] = stats.mode(ground_truth_data[k])

    # Our labels for each color chip 

    labels = np.array([chip[1].mode[0] for chip in ground_truth.items()])    
    # print (labels)

    # All unique color terms from the ground truth. These determine our "categories"
    color_terms = list(set(labels))

    print ("Color categories:", color_terms)

    # Construct feature matrix
    F = [indexCoord[k] for k in range(1,331)]

    # print (np.array(F))

    # Lightness variable is a letter, lets convert this into an integer.
    F = np.transpose(F)
    # print (F[0])
    light_num = [int(ord(letter)-64) for letter in F[0]]
    F = np.transpose([light_num, F[1]])

    terms_inds = {}

    # Convert coordinates into tuple(int,int) THIS WILL ONLY WORK FOR MUNSELL SPACE, IF YOU WANT TO DO CIELAB, YOU NEED TO MODIFY
    F = [tuple([int(coord[0]), int(coord[1])]) for coord in F]

    # print (F)

    for ind, term in enumerate(labels):
        if term not in terms_inds:
            terms_inds[term] = [F[ind]]
        else:
            terms_inds[term].append(F[ind])

    # print (terms_inds)

    accuracy = LOOCV_Exemplar(F, terms_inds)
    acc_list.append(accuracy)
    
    
    print ("Accuracy for language", i, ":",accuracy)



    #for term in color_terms:
    #    print (term)
    #    print (np.array([spatial.distance.euclidean(coord, (5,29)) for coord in terms_inds[term] if coord != (5,29)]))
        
print ("EXEMPLAR MODEL AVERAGE OVERALL ACCURACY:", np.mean(acc_list))
        